## Задача. Построить модель прогнозирования факта сдачи теста

<img src="image/images2.png" alt="Drawing" style="width: 380px;" align="left"/> <br />

### Загружаем библиотеки

In [1]:
import itertools
import numpy as np
import pandas as pd
import scipy

from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import Ridge

import matplotlib.pyplot as plt
%matplotlib inline 

### Загружаем данные

In [2]:
df = pd.read_csv('dataset.csv', sep=';')
print(df.shape)
df.head()

(2133, 12)


,Unnamed: 0,Unnamed: 0.1,school,school_setting,school_type,classroom,teaching_method,n_student,gender,lunch,posttest,target
0,0,0,ANKYI,Urban,Non-public,6OL,Standard,20.0,Male,Does not qualify,72.0,1.0
1,1,1,ANKYI,Urban,Non-public,6OL,Standard,20.0,Male,Does not qualify,79.0,0.0
2,2,2,ANKYI,Urban,Non-public,6OL,Standard,NaN,Male,Does not qualify,76.0,1.0
3,3,3,ANKYI,Urban,Non-public,6OL,Standard,20.0,Male,Does not qualify,77.0,1.0
4,4,4,ANKYI,Urban,Non-public,6OL,Standard,20.0,Male,Does not qualify,76.0,0.0


### Описание полей
* **school** - Код школы, в которой учится ученик  
* **school_setting** - Тип района в котором находится школа  
* **school_type** - Частная или нет школа  
* **classroom** - внутренний код класса, в котором проиходят занятия  
* **teaching_method** - является ли обучение экспериментальным  
* **n_student** - возраст ученика  
* **gender** - пол ученика  
* **lunch** - Является ли ученик слабозащищенной категорией населения  
* **posttest** - балл по тесту
* **target** - сдал или нет. 1- сдал.

### Разобьем выборку на train / valid / test

In [3]:
X = df.drop(['posttest','Unnamed: 0','Unnamed: 0.1'], axis=1)
X = X.drop(['target'], axis=1)
y = df['posttest']

X_tv, X_test, y_tv, y_test = train_test_split(X, y, random_state=1234)
X_train, X_valid, y_train, y_valid = train_test_split(X_tv, y_tv, test_size=0.5,random_state=1234)

X_train.shape, X_valid.shape, X_test.shape

((799, 8), (800, 8), (534, 8))

У меня нет больше информации кроме как, что target это «сдал или нет», а posttest это баллы по тесту. То есть я могу дать предположение относительно данных, что target это сдача предмета, а posttest это сдача определенного теста (накопительной части для сдачи предмета). В связи с тем, что у отмеченных этих двух признаков нет корреляции (околонулевая), я могу поступить относительно модели двумя способами. Первый – это убрать признак target. Либо оставить данный признак в связи с тем, что он может послужить хорошим признаком. Руководствуясь логикой, что это некое заглядывание в будущее, в рамках обобщения модели, правильнее будет убрать данный признак.

### Заполнение пропусков

In [4]:

X_train = X_train.fillna(X_train.mean())
X_valid = X_valid.fillna(X_valid.mean())
X_test = X_test.fillna(X_train.mean())


Тестовая выборка должна быть заполнена по трейну. В валидационной выборке я решил заполнить пропуски по ней же в рамках избежания переобучения на трейне при нахождении woe.

-----------

В качестве целевой переменной  будем рассматривать балл по тесту.  <br />
Построим линейную регрессию.

### Заинжинирим новую переменную
пол*возраст
для каждой комбинации посчитаем weight of evidence и будем его использовать в модели

In [5]:
X_valid=X_valid.reset_index()

X_train['woe_agegender']=np.mean(y_valid)
X_test['woe_agegender']=np.mean(y_valid)

for i in X_valid.gender.unique():
    for j in X_valid.n_student.unique():
        
        l=X_valid.loc[(X_valid.gender==i) & (X_valid.n_student>=j)]
        woe=y_valid.loc[l['index'].tolist()].mean()
        X_train.loc[(X_train.gender==i) & (X_train.n_student>=j), 'woe_agegender'] = woe
        
        
        X_test.loc[(X_test.gender==i) & (X_test.n_student>=j), 'woe_agegender'] = woe

In [6]:
X_train['woe_agegender'].unique()

array([67.36349454, 55.45454545, 64.69736842, 67.41705426, 64.96774194])

Нахожу значения woe относительно пола и количества студентов на основе валидационной выборки и эти значения при тех же значениях пола и колчичества студентов добавляю в трейин и тест. (В случае если нет совпадения, я решил заполнить средним значением)

### Дальше накинем one-hot encoding и маштабирование

Помимо этого добавил регулизацию из-за потенциально большого появления новых признаков и решения проблемы мультиколиниарности из-за отсутсвия выкинутых первых столбцов после One-hot encoding

Помимо этого на будущее добавил возможность выкинуть не нужные столбцы для улучшения модели

In [25]:
class LinearRegressionEncoder(BaseEstimator):
    def __init__(self, list_drop=None):
        self.linreg = Ridge(0.1)
        self.encoders = {}
        self.list_drop=list_drop
    
    
    def fit(self, X, y):
        X_enc = X.copy()
        
        categorical_features = X.columns[X.dtypes == 'object']
        numerical_features=X._get_numeric_data().columns
        
        
        le = ColumnTransformer( [("scaler", StandardScaler(),numerical_features),
                         ('encoder',OneHotEncoder(handle_unknown='ignore'),categorical_features )], remainder = 'passthrough')
       
        X_enc = pd.DataFrame(le.fit_transform(X).toarray())
        self.encoders = le
 
        
        if self.list_drop != None:
            X_enc=X_enc.drop(self.list_drop, axis=1)
        
        self.col=X_enc.columns
        self.linreg.fit(X_enc, y)
        
        # считаем значимость
        y_pred = self.linreg.predict(X_enc)
        sse = np.sum((y_pred - y) ** 2, axis=0) / float(X_enc.shape[0] - X_enc.shape[1])
        
        se = np.array([np.sqrt(np.diag(sse * np.linalg.pinv(np.dot(X_enc.T, X_enc))))])
        print(np.shape(se))
        self.t = self.linreg.coef_ / se
        
        self.p = 2 * (1 - scipy.stats.t.cdf(np.abs(self.t), y.shape[0] - X_enc.shape[1]))
        
        return self
    
    def predict(self, X):
        X_enc = X.copy()

        X_enc = pd.DataFrame(self.encoders.transform(X).toarray())
        
        if self.list_drop != None:
            X_enc=X_enc.drop(self.list_drop, axis=1)
            
        y_pred = self.linreg.predict(X_enc)
        
        return y_pred
    
    def score(self, X, y):        
        
        y_pred = self.predict(X)
        return mean_absolute_error(y, y_pred)

In [26]:
%%time
linreg = LinearRegressionEncoder()
linreg.fit(X_train, y_train)

(1, 133)
Wall time: 42.1 ms


LinearRegressionEncoder()

In [27]:
print('TRAIN MSE:', linreg.score(X_train, y_train))
print('TEST MSE:', linreg.score(X_test, y_test))

TRAIN MSE: 2.344137915535318
TEST MSE: 2.4837855206964172


### Ошибка на тесте не сильно больше чем на трейне. Значит модель отличная

Улучшим модель, исключив из данных незначимые факторы

In [28]:
## здесь дописать
for p_val, factor in itertools.zip_longest(np.array(linreg.p[0]), np.array(linreg.col)):
    print(factor, ' - ', round(p_val, 6))

0  -  0.505874
1  -  0.968184
2  -  0.173256
3  -  0.074622
4  -  8.9e-05
5  -  0.305676
6  -  0.0
7  -  0.336509
8  -  6e-06
9  -  0.905707
10  -  0.696825
11  -  0.0
12  -  0.0
13  -  0.0
14  -  0.0
15  -  0.0
16  -  0.467986
17  -  0.560886
18  -  0.0
19  -  0.305137
20  -  0.017251
21  -  0.63097
22  -  0.004494
23  -  0.0
24  -  0.150208
25  -  0.00142
26  -  0.0
27  -  0.206562
28  -  0.000257
29  -  0.203221
30  -  0.720272
31  -  0.0
32  -  0.0
33  -  0.998736
34  -  0.625553
35  -  0.115634
36  -  1.3e-05
37  -  0.885414
38  -  1e-06
39  -  0.000637
40  -  0.002322
41  -  0.080054
42  -  0.0
43  -  4e-06
44  -  0.0
45  -  0.209981
46  -  0.106492
47  -  0.0
48  -  0.00052
49  -  0.00011
50  -  0.029741
51  -  0.000828
52  -  0.04539
53  -  0.0
54  -  0.146075
55  -  0.001511
56  -  1e-05
57  -  0.028396
58  -  0.0
59  -  0.727205
60  -  0.24887
61  -  0.643046
62  -  0.674529
63  -  9.3e-05
64  -  0.884901
65  -  0.0
66  -  0.0
67  -  0.021957
68  -  0.000208
69  -  0.065542
7

удалим факторы у которых pvalue = 0 после округление до 10 знака и перевзвесим коэффициенты

In [29]:
sign = [f for p, f in itertools.zip_longest(np.array(linreg.p[0]), np.array(linreg.col)) if np.round(p,10) == 0]
sign

[11, 12, 13, 14, 18, 23, 26, 32, 44, 58, 65, 77, 89, 95, 114]

In [30]:
%%time
linreg2 = LinearRegressionEncoder(list_drop=sign)
linreg2.fit(X_train, y_train)

(1, 118)
Wall time: 44.4 ms


LinearRegressionEncoder(list_drop=[11, 12, 13, 14, 18, 23, 26, 32, 44, 58, 65,
                                   77, 89, 95, 114])

In [31]:
print('TRAIN MSE after drop insignificant:', 
      linreg2.score(X_train, y_train))
print('TEST MSE after drop insignificant', 
      linreg2.score(X_test, y_test))

TRAIN MSE after drop insignificant: 2.6217485246126833
TEST MSE after drop insignificant 2.695492480394026


### После удаления получили модель чуть похуже, но в ней оценки коэффициентов несмещены и переобучение стало немного меньше (разница между тестом и трейном меньше), а значит модель получилась более обобщенная